# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
!pip install PTable

  Running setup.py bdist_wheel for PTable ... done
  Stored in directory: /root/.cache/pip/wheels/22/cc/2e/55980bfe86393df3e9896146a01f6802978d09d7ebcba5ea56
Successfully built PTable


In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [7]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity 
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Schema

For the first query: 
- sessionId: int
- itemInSession: int
- artist: text
- song: text
- length: float
- Primary key = (sessionId, itemInSession)

For the second query:
- userId : int
- sessionId : int
- artist: text
- song: text
- itemInSession: int
- firstName: text
- lastName: text
- Primary key = (userId, sessionId) with clustering column itemInSession

For the third query:
- song : text
- userId : int
- firstName: text
- lastName: text
- Primary key = (song) with clustering column userId

In [9]:
# CREATE TABLE FOR QUERY 1
query = "CREATE TABLE IF NOT EXISTS songs "
query = query + "(itemInSession int, sessionId int, artist text, song text, length float, PRIMARY KEY(itemInSession, sessionId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [10]:
# Insert new entry into table 1
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs(itemInSession, sessionId, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[3]), int(line[8]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
## Perform query 1
query = "select artist, song, length from songs where sessionId = 338 and itemInSession = 4 ALLOW FILTERING;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

## Show the result of query 1

In [12]:
query_1 = PrettyTable(['Artist', 'Song', 'Length'])
for row in rows:
    query_1.add_row([row.artist, row.song, row.length])
print(query_1)

+-----------+---------------------------------+--------------------+
|   Artist  |               Song              |       Length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [13]:
# CREATE TABLE FOR QUERY 2
query = "CREATE TABLE IF NOT EXISTS artists "
query = query + "(userId int, sessionId int, itemInSession int, artist text, firstName text, lastName text,song text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query) 
except Exception as e:
    print(e)

In [14]:
# Insert new entry into table 2
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artists(itemInSession, sessionId, userId, artist, firstName, lastName, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[3]), int(line[8]), int(line[10]), line[0], line[1], line[4], line[9]))

In [15]:
# Perform query 2
query = "select artist, song, firstName, lastName from artists where sessionId = 182 and userId = 10 ALLOW FILTERING;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [16]:
# Show data of query 2
query_2 = PrettyTable(['Artist', 'Song', 'FirstName', 'lastName'])
for row in rows:
    query_2.add_row([row.artist, row.song, row.firstname, row.lastname])
print(query_2)

+-------------------+------------------------------------------------------+-----------+----------+
|       Artist      |                         Song                         | FirstName | lastName |
+-------------------+------------------------------------------------------+-----------+----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie  |   Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie  |   Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie  |   Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie  |   Cruz   |
+-------------------+------------------------------------------------------+-----------+----------+


In [19]:
# CREATE TABLE FOR QUERY 3
query = "CREATE TABLE IF NOT EXISTS users"
query = query + "(userId int, song text, firstName text, lastName text, PRIMARY KEY ((song), userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [20]:
# Insert new entry into table 3
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users(userId, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line[10]), line[9], line[1], line[4]))

In [21]:
# Perform query 3
query = "select firstName, lastName from users where song = 'All Hands Against His Own' ALLOW FILTERING;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [22]:
# Show data of query 3
query_3 = PrettyTable(['FirstName', 'LastName'])
for row in rows:
    query_3.add_row([row.firstname, row.lastname])
print(query_3)

+------------+----------+
| FirstName  | LastName |
+------------+----------+
| Jacqueline |  Lynch   |
|   Tegan    |  Levine  |
|    Sara    | Johnson  |
+------------+----------+


### Drop the tables before closing out the sessions

In [23]:
session.execute("DROP TABLE IF EXISTS songs")
session.execute("DROP TABLE IF EXISTS users")
session.execute("DROP TABLE IF EXISTS artists")

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()